# TF MME W/ Boto3

<b>Setting</b>: conda_tf2_p38 & Classic Notebook Instances

## Local Model Training

In [5]:
%%writefile local_tf.py
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


#ANN Model
def createModel():
    model = Sequential()
    model.add(Dense(13, input_shape=(13,), activation='relu'))
    model.add(Dense(28, activation='relu'))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

#Load data
boston = datasets.load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['MEDV'] = boston.target 

#Split Model
X = df.drop(['MEDV'], axis = 1) 
y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

#minimum epochs 15
model = createModel()
monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience=15)
model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[monitor_val_acc], epochs=5)
model.save('0000001')

Writing local_tf.py


In [6]:
!python local_tf.py

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include t

## Package for SageMaker

### Script for TF Serving to control pre/post processing

In [7]:
%%writefile inference.py

import json

def input_handler(data, context):
    """ Pre-process request input before it is sent to TensorFlow Serving REST API
    Args:
        data (obj): the request data, in format of dict or string
        context (Context): an object containing request and configuration details
    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """
    if context.request_content_type == 'application/json':
        # pass through json (assumes it's correctly formed)
        d = data.read().decode('utf-8')
        print("------------")
        print(d)
        print(type(d))
        print("------------")
        return d if len(d) else ''

    if context.request_content_type == 'text/csv':
        # very simple csv handler
        return json.dumps({
            'instances': [float(x) for x in data.read().decode('utf-8').split(',')]
        })

    raise ValueError('{{"error": "unsupported content type {}"}}'.format(
        context.request_content_type or "unknown"))


def output_handler(data, context):
    """Post-process TensorFlow Serving output before it is returned to the client.
    Args:
        data (obj): the TensorFlow serving response
        context (Context): an object containing request and configuration details
    Returns:
        (bytes, string): data to return to client, response content type
    """
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))

    response_content_type = context.accept_header
    print("-------")
    print(data)
    print(type(data))
    print("-----")
    prediction = data.content
    return prediction, response_content_type

Overwriting inference.py


In [9]:
#Setup
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess
import shlex

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
#role

us-east-1


In [10]:
#Copy inference.py into a code file for sm to recognize
createDirectory = "mkdir code"
copyInference = "cp inference.py code"
p1 = subprocess.call(createDirectory, shell=True)
p2 = subprocess.call(copyInference, shell=True)

In [11]:
#Tar file with model artifacts
bashCommand = "tar -cvpzf model.tar.gz ./0000001"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [12]:
#Tar ball for inference code, inference.py is consistent across all models in an MME
bashCommand = "tar -cvpzf source.tar.gz inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [13]:
#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

sagemaker-us-east-1-474422712127


### Make Copies Of Model Data In S3

In [39]:
%%sh

s3_bucket='sagemaker-us-east-1-474422712127' #replace this with your default bucket

for i in {0..5}
do
  aws s3 cp model.tar.gz s3://$s3_bucket/mme-tensorflow-boto3/tf-$i.tar.gz 
done

upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-0.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-1.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-2.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-3.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-4.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/tf-5.tar.gz


In [15]:
!aws s3 ls s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/ #replace with your default bucket value for this cell and following

2022-08-04 18:48:08      24348 tf-0.tar.gz
2022-08-04 18:48:09      24348 tf-1.tar.gz
2022-08-04 18:48:09      24348 tf-2.tar.gz
2022-08-04 18:48:10      24348 tf-3.tar.gz
2022-08-04 18:48:10      24348 tf-4.tar.gz
2022-08-04 18:48:11      24348 tf-5.tar.gz


In [16]:
!aws s3 cp source.tar.gz s3://sagemaker-us-east-1-474422712127/mme-tf-inf-script/source.tar.gz

upload: ./source.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-tf-inf-script/source.tar.gz


In [17]:
!aws s3 ls s3://sagemaker-us-east-1-474422712127/mme-tf-inf-script/

2022-08-04 18:50:59        793 source.tar.gz


In [18]:
source_dir = 's3://sagemaker-us-east-1-474422712127/mme-tf-inf-script/'

In [22]:
!aws s3 ls {source_dir}

2022-08-04 18:50:59        793 source.tar.gz


In [19]:
s3_bucket='sagemaker-us-east-1-474422712127'
model_url = 's3://{}/mme-tensorflow-boto3/'.format(s3_bucket) ## MODEL S3 URL
model_url

's3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/'

In [20]:
!aws s3 ls {model_url}

2022-08-04 18:48:08      24348 tf-0.tar.gz
2022-08-04 18:48:09      24348 tf-1.tar.gz
2022-08-04 18:48:09      24348 tf-2.tar.gz
2022-08-04 18:48:10      24348 tf-3.tar.gz
2022-08-04 18:48:10      24348 tf-4.tar.gz
2022-08-04 18:48:11      24348 tf-5.tar.gz


### Retrieve Container

In [21]:
# retrieve tf image
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region="us-east-1",
    version="2.3.0",
    py_version="py3",
    instance_type="ml.m5.xlarge",
    image_scope="inference"
)
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu'

### SageMaker Model Entity Creation

API Call: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model

In [23]:
from time import gmtime, strftime
model_name = 'mme-tf' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "MultiModel",
            "ModelDataUrl": model_url,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': source_dir,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: mme-tf2022-08-04-18-55-18
Model data Url: s3://sagemaker-us-east-1-474422712127/mme-tensorflow-boto3/
Model Arn: arn:aws:sagemaker:us-east-1:474422712127:model/mme-tf2022-08-04-18-55-18


### Endpoint Config Creation

API Call: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config

In [24]:
#Step 2: EPC Creation
tf_epc_name = "mme-tf" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=tf_epc_name,
    ProductionVariants=[
        {
            "VariantName": "tfvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint-config/mme-tf2022-08-04-18-56-48


### Endpoint Creation

API Call: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint

In [25]:
#Step 3: EP Creation
endpoint_name = "mme-tf" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=tf_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-tf2022-08-04-18-56-52


In [26]:
#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'mme-tf2022-08-04-18-56-52', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-tf2022-08-04-18-56-52', 'EndpointConfigName': 'mme-tf2022-08-04-18-56-48', 'ProductionVariants': [{'VariantName': 'tfvariant', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu', 'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:91ebb7428846c5f7b515d5d9b8389a14c73d0c5d02657f4a6413592124333278', 'ResolutionTime': datetime.datetime(2022, 8, 4, 18, 56, 56, 660000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': da

### Invoke MME

In [29]:
import boto3
import json
from sagemaker.serializers import JSONSerializer
import pandas as pd

test = df[:1]
testX = test.drop("TARGET", axis=1)
testX = testX[:1].values.tolist()
sampInput = {"inputs": testX}
print(sampInput)

{'inputs': [[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]]}


In [32]:
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
jsons = JSONSerializer()
content_type = "application/json"
payload = jsons.serialize(sampInput)
response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload,
        ContentType=content_type,
        TargetModel = "tf-1.tar.gz")
result = json.loads(response['Body'].read().decode())['outputs']
result

[[20.5638695]]

In [37]:
for i in range (0,6):
    target_model = "tf-{}.tar.gz".format(i)
    print(target_model)
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload,
        ContentType=content_type,
        TargetModel = target_model)
    print(json.loads(response['Body'].read().decode())['outputs'])

tf-0.tar.gz
[[20.5638695]]
tf-1.tar.gz
[[20.5638695]]
tf-2.tar.gz
[[20.5638695]]
tf-3.tar.gz
[[20.5638695]]
tf-4.tar.gz
[[20.5638695]]
tf-5.tar.gz
[[20.5638695]]
